## Hacky Boilerplates
- Install `feast` with pip.
- Activate user service account with credentials JSON.
- Hacks to retrieve essential information for deployments and serving.

**NOTE**: This code block might hangs for a long time.

In [ ]:
import json
import os
import re
import subprocess

from googleapiclient import discovery
from oauth2client.client import GoogleCredentials

# Install dependencies
_ = subprocess.call(['pip', 'install', 'feast'], shell=True)
# Retrieve user service account.
_ = subprocess.call(['gcloud auth activate-service-account --key-file=$GOOGLE_APPLICATION_CREDENTIALS'],
                    shell=True)

# Create KUBECONFIG. Use credential file to retrieve project/deployment names.
cred_path = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')
cred = {}
with open(cred_path, 'r') as c:
    cred = json.load(c)

PROJECT = cred['project_id']
APP_NAME = re.search('([a-z\-]+)-user'.format(PROJECT),
                     cred['client_email']).group(1)

p = subprocess.Popen(['gcloud', 'container', 'clusters', 'list',
                      '--filter', 'name=%s' % APP_NAME, '--format', 'json'],
                    stdout=subprocess.PIPE)
out, _ = p.communicate()
config = json.loads(out)[0]
ZONE = config['zone']

print('PROJECT =', PROJECT)
print('APP_NAME =', APP_NAME)
print('ZONE =', ZONE)

## Linear Model

In [ ]:
import numpy as np

train_data = np.random.randint(1, high=100, size=(200, 100))

A = np.insert(train_data, 100, 1, axis=1)
Y = np.random.randint(1, high=100, size=200)

x = np.linalg.lstsq(A, Y, rcond=0)[0]
m, b = x[:100], x[100]

## Local Prediction

In [ ]:
def local_predict(x):
    return m * x + b

feature = np.random.randint(1, high=100, size=100)
p = local_predict(feature)
np.set_printoptions(precision=3)
print(p)

## Save the model

In [ ]:
import json
import os

MODEL_FILE = 'simple_model.dat'

model = {
    'm': m.tolist(),
    'b': b,
}

model_path = os.path.join(os.getenv('HOME', '/home'), MODEL_FILE)
print('writing to', model_path)

with open(model_path, 'w+') as f:
    json.dump(model, f)

## Deploy with Kubeflow

In [ ]:
import fairing
import sys
import importlib

DOCKER_REGISTRY = 'gcr.io/{}/fairing-job'.format(PROJECT)
BASE_IMAGE = 'gcr.io/kubeflow-images-public/fairing-base:v20190510'
SERVING_LABEL = 'kubeflow-fairing-demo'

print('docker registry:', DOCKER_REGISTRY)
print('base image:', BASE_IMAGE)

In [ ]:
import deploy_with_fairing
import uuid

# To disambiguate between different deployments.
serving_label = SERVING_LABEL + '-' + uuid.uuid4().hex[:4]
print('Deploying service with selector', serving_label)

# Register for docker credential. Needed for docker image pushes.
_ = subprocess.call(['gcloud auth configure-docker --quiet'], shell=True)

importlib.reload(deploy_with_fairing)
deploy_with_fairing.deploy(DOCKER_REGISTRY, BASE_IMAGE, serving_label=serving_label)

In [ ]:
import subprocess
from kubernetes import client, config

# Need to set up KUBECONFIG. Kubernetes API client depends on it.
subprocess.call(['gcloud', 'container', 'clusters', 'get-credentials', APP_NAME,
                 '--zone', ZONE, '--project', PROJECT])
config.load_kube_config()
c = client.Configuration()
client.Configuration.set_default(c)

v1 = client.CoreV1Api()
body = client.V1Service()
label_selector = 'serving=%s' % serving_label
resp = v1.list_service_for_all_namespaces(label_selector=label_selector)

service_name = resp.items[0].metadata.name
namespace = resp.items[0].metadata.namespace

print('fairing service: {0}/{1}'.format(namespace, service_name))

## Serving with Kubeflow

In [ ]:
import json
import requests

def predict(url, data, feature_names=None):
    pdata={
        "data": {
            "names":feature_names,
            "tensor": {
                "shape": np.asarray(data.shape).tolist(),
                "values": data.flatten().tolist(),
            },
        }
    }
    serialized_data = json.dumps(pdata)
    r = requests.post(url, data={'json':serialized_data})
    return r

def extract_prediction_array(content):
    c = json.loads(content)
    return np.array(c.get('data', {}).get('tensor', {}).get('values'))

In [ ]:
import numpy as np
import pprint

url = "http://{service_name}.{namespace}.svc.cluster.local:5000/predict".format(
    service_name=service_name,
    namespace=namespace)

data = np.random.randint(1, high=100, size=100)
r = predict(url, data)

prediction = extract_prediction_array(r.content)
print('prediction:')
pprint.pprint(prediction)